In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
from sklearn.preprocessing import StandardScaler
import warnings

In [2]:
df_recent = pd.read_csv('~/Desktop/Redback/Ultimate_df_recent.csv')
remove_cols = ['Unnamed: 0'] 
df_recent = df_recent.drop(remove_cols, axis=1)

In [87]:
input_params = df_recent.iloc[:, -33:]

In [3]:
train = df_recent.sample(frac = 0.7, random_state = 1)
test = df_recent[df_recent.index.isin(train.index) == False].copy()

In [4]:
train

,SpeedKMH_mean,SpeedKMH_max,SpeedKMH_min,Gas_mean,Gas_max,Gas_min,Brake_mean,Brake_max,Brake_min,Clutch_mean,...,suspensions_REAR_STATIC_CAMBER,suspensions_REAR_SPRING_RATE,suspensions_REAR_BUMP_STOP_RATE,suspensions_REAR_PACKER_RANGE,suspensions_REAR_DAMP_BUMP,suspensions_REAR_DAMP_FAST_BUMP,suspensions_REAR_DAMP_REBOUND,suspensions_REAR_DAMP_FAST_REBOUND,tyres_FRONT_PRESSURE_STATIC,tyres_REAR_PRESSURE_STATIC
1160,56.220621,80.458206,36.446529,0.752850,1.0,2.382579e-05,0.034343,0.864560,1.401298e-45,1.0,...,-0.12138,49675.84259,188429.06769,0.04424,1511.81166,1003.28781,4195.36159,2068.46603,13.96513,8.68237
571,59.936472,91.717621,40.058083,0.814326,1.0,2.423351e-04,0.053950,0.961056,1.245334e-41,1.0,...,-2.35767,56648.86635,124979.65621,0.08939,1161.13758,3342.89845,3747.30276,3478.35963,9.18841,14.31058
65,60.443585,92.711037,39.924118,0.807149,1.0,8.702921e-05,0.047823,0.724156,3.525527e-41,1.0,...,-0.58362,41686.70293,70652.92167,0.11752,1571.26420,790.08340,4475.43916,3158.38842,15.86859,19.85950
115,61.703306,92.497635,39.986065,0.768248,1.0,6.858435e-10,0.063544,0.999464,1.057420e-41,1.0,...,-2.59556,37483.25602,64155.59613,0.07298,3999.25314,3363.71093,1933.52925,2256.09291,10.01387,19.44935
1253,61.056494,89.641335,39.089828,0.807825,1.0,2.768625e-04,0.053682,0.992410,1.656335e-42,1.0,...,-0.61541,63465.21155,137675.76501,0.10888,3664.33695,3461.12177,3234.86586,2838.43670,17.79766,11.45303
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,60.200013,91.442917,39.668159,0.816757,1.0,7.403053e-05,0.063082,0.975947,2.072520e-42,1.0,...,-0.20239,59369.22920,178725.92006,0.10414,1314.70925,2362.81597,3093.43455,3441.95272,8.46643,16.94134
1128,59.250398,90.414803,36.961681,0.846792,1.0,1.489518e-04,0.041468,0.756643,3.203368e-42,1.0,...,-2.47687,34792.67388,119161.99136,0.11152,2466.65030,711.89177,3516.85544,2707.87679,9.18703,14.75114
333,59.222145,84.660400,39.724865,0.805181,1.0,1.358520e-04,0.035524,0.890022,1.401298e-45,1.0,...,-2.10185,64460.62646,139018.76963,0.06611,2243.25365,623.18994,1818.47609,2244.12883,7.88934,19.58432
153,53.304301,74.743004,34.967068,0.698706,1.0,1.199077e-04,0.018373,0.738442,1.401298e-45,1.0,...,-0.08570,47267.90840,138973.85153,0.08512,2947.84794,3159.16189,3485.33395,689.02856,17.08241,8.13701


In [89]:
response = ['LapTime_max']
predictors = input_params.columns
mu = train[predictors].mean() # mean for each feature
sigma = train[predictors].std() # std for each feature

In [90]:
mu.head()

brakes_DATA_MAX_TORQUE             611.050241
brakes_DATA_FRONT_SHARE              0.676043
drivetrain_DIFFERENTIAL_POWER        0.414661
drivetrain_DIFFERENTIAL_COAST        0.469073
drivetrain_DIFFERENTIAL_PRELOAD     12.210391
dtype: float64

In [91]:
sigma.head()

brakes_DATA_MAX_TORQUE             112.753669
brakes_DATA_FRONT_SHARE              0.095848
drivetrain_DIFFERENTIAL_POWER        0.291044
drivetrain_DIFFERENTIAL_COAST        0.274821
drivetrain_DIFFERENTIAL_PRELOAD      6.666559
dtype: float64

In [92]:
train[predictors] = (train[predictors] - mu)/sigma
test[predictors] = (test[predictors] - mu)/sigma

In [93]:
mu = train[predictors].mean() # mean for each feature
sigma = train[predictors].std() # std for each feature

In [94]:
mu.head()

brakes_DATA_MAX_TORQUE             1.245558e-17
brakes_DATA_FRONT_SHARE            1.234060e-15
drivetrain_DIFFERENTIAL_POWER      2.299491e-17
drivetrain_DIFFERENTIAL_COAST     -1.303045e-16
drivetrain_DIFFERENTIAL_PRELOAD   -1.053933e-16
dtype: float64

In [95]:
sigma.head()

brakes_DATA_MAX_TORQUE             1.0
brakes_DATA_FRONT_SHARE            1.0
drivetrain_DIFFERENTIAL_POWER      1.0
drivetrain_DIFFERENTIAL_COAST      1.0
drivetrain_DIFFERENTIAL_PRELOAD    1.0
dtype: float64

In [96]:
y_train = train.iloc[:, train.columns == "LapTime_max"]
X_train = train.iloc[:, -33:]
y_test = test.iloc[:, test.columns == "LapTime_max"]
X_test = test.iloc[:, -33:]

In [97]:
train = y_train.join(X_train)
test = y_test.join(X_test)

In [99]:
test

,LapTime_max,brakes_DATA_MAX_TORQUE,brakes_DATA_FRONT_SHARE,drivetrain_DIFFERENTIAL_POWER,drivetrain_DIFFERENTIAL_COAST,drivetrain_DIFFERENTIAL_PRELOAD,suspensions_BASIC_WHEELBASE,suspensions_BASIC_CG_LOCATION,suspensions_ARB_FRONT,suspensions_ARB_REAR,...,suspensions_REAR_STATIC_CAMBER,suspensions_REAR_SPRING_RATE,suspensions_REAR_BUMP_STOP_RATE,suspensions_REAR_PACKER_RANGE,suspensions_REAR_DAMP_BUMP,suspensions_REAR_DAMP_FAST_BUMP,suspensions_REAR_DAMP_REBOUND,suspensions_REAR_DAMP_FAST_REBOUND,tyres_FRONT_PRESSURE_STATIC,tyres_REAR_PRESSURE_STATIC
15,41301,-0.947408,-0.186581,-0.987344,-1.492070,0.862829,0.987832,1.808506,1.598770,-0.989155,...,0.450333,-0.436838,-0.842743,-0.243007,0.456334,1.484082,-1.404045,0.728789,1.636737,-0.200990
20,41202,-0.947408,-0.186581,-0.987344,-1.492070,0.862829,0.987832,1.808506,1.598770,-0.989155,...,0.450333,-0.436838,-0.842743,-0.243007,0.456334,1.484082,-1.404045,0.728789,1.636737,-0.200990
21,41136,-0.947408,-0.186581,-0.987344,-1.492070,0.862829,0.987832,1.808506,1.598770,-0.989155,...,0.450333,-0.436838,-0.842743,-0.243007,0.456334,1.484082,-1.404045,0.728789,1.636737,-0.200990
24,42048,-0.459390,1.286896,-0.649045,0.364408,0.701657,0.306738,0.594878,-1.763979,0.233039,...,1.567738,1.219924,-1.185068,-0.953711,0.627447,1.089141,-0.158915,0.339060,1.223062,0.435088
25,41922,-0.459390,1.286896,-0.649045,0.364408,0.701657,0.306738,0.594878,-1.763979,0.233039,...,1.567738,1.219924,-1.185068,-0.953711,0.627447,1.089141,-0.158915,0.339060,1.223062,0.435088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1312,42156,-0.527661,0.633884,-1.000572,0.826381,-0.259713,1.698538,-0.764420,-0.964367,0.886356,...,-0.470729,-0.751698,-0.995565,-0.677469,0.007018,1.443360,-1.284856,-0.510023,0.502647,0.734029
1319,42354,-0.527661,0.633884,-1.000572,0.826381,-0.259713,1.698538,-0.764420,-0.964367,0.886356,...,-0.470729,-0.751698,-0.995565,-0.677469,0.007018,1.443360,-1.284856,-0.510023,0.502647,0.734029
1320,42117,-0.527661,0.633884,-1.000572,0.826381,-0.259713,1.698538,-0.764420,-0.964367,0.886356,...,-0.470729,-0.751698,-0.995565,-0.677469,0.007018,1.443360,-1.284856,-0.510023,0.502647,0.734029
1322,42282,-0.527661,0.633884,-1.000572,0.826381,-0.259713,1.698538,-0.764420,-0.964367,0.886356,...,-0.470729,-0.751698,-0.995565,-0.677469,0.007018,1.443360,-1.284856,-0.510023,0.502647,0.734029


In [100]:
train

,LapTime_max,brakes_DATA_MAX_TORQUE,brakes_DATA_FRONT_SHARE,drivetrain_DIFFERENTIAL_POWER,drivetrain_DIFFERENTIAL_COAST,drivetrain_DIFFERENTIAL_PRELOAD,suspensions_BASIC_WHEELBASE,suspensions_BASIC_CG_LOCATION,suspensions_ARB_FRONT,suspensions_ARB_REAR,...,suspensions_REAR_STATIC_CAMBER,suspensions_REAR_SPRING_RATE,suspensions_REAR_BUMP_STOP_RATE,suspensions_REAR_PACKER_RANGE,suspensions_REAR_DAMP_BUMP,suspensions_REAR_DAMP_FAST_BUMP,suspensions_REAR_DAMP_REBOUND,suspensions_REAR_DAMP_FAST_REBOUND,tyres_FRONT_PRESSURE_STATIC,tyres_REAR_PRESSURE_STATIC
1160,43914,1.513706,-0.496028,0.749747,-0.574274,0.945005,0.443342,-0.995821,-1.910269,-1.457874,...,0.948846,0.322634,1.654680,-1.664843,-1.170158,-1.054219,1.388344,-0.019542,0.373114,-1.278439
571,41256,0.440057,1.564940,-0.589535,0.314667,-0.189615,1.628311,0.317802,0.931124,1.298159,...,-1.278742,0.854476,0.118775,0.265861,-1.541565,1.354502,0.884019,1.675101,-0.908377,0.292293
65,40647,1.250806,0.863518,-0.374172,1.551179,-0.634678,-1.173899,-1.213966,-0.403501,0.423389,...,0.488404,-0.286708,-1.196300,1.468756,-1.107191,-1.273721,1.703593,1.290507,0.883771,1.840896
115,39945,-1.830338,0.096889,-0.169427,-1.246456,1.638991,-0.223908,1.382735,-0.645199,0.370391,...,-1.515706,-0.607310,-1.353579,-0.435863,1.464349,1.375929,-1.157524,0.205979,-0.686924,1.726430
1253,40476,-1.616643,1.317361,1.204248,1.282240,1.104064,-1.093954,-0.402191,0.170541,0.035663,...,0.456738,1.374368,0.426107,1.099293,1.109631,1.476217,0.307232,0.905936,1.401298,-0.505198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,40752,-1.253020,-0.787218,0.917728,0.787556,-1.069729,1.388933,1.446852,-1.307422,-0.208520,...,0.868151,1.061962,1.419798,0.896601,-1.378914,0.345468,0.148040,1.631341,-1.102068,1.026490
1128,41544,1.592009,-0.186163,-1.186386,0.152926,0.913135,-1.241835,-0.814562,-1.027952,1.394277,...,-1.397478,-0.812524,-0.022052,1.212184,-0.158866,-1.354222,0.624633,0.749008,-0.908747,0.415245
333,41787,0.102421,0.694292,-0.802115,-0.297512,-0.140659,0.586363,-0.215168,-0.543295,1.241881,...,-1.023917,1.450289,0.458616,-0.729638,-0.395471,-1.445544,-1.287025,0.191599,-1.256889,1.764098
153,46311,1.094944,-1.777535,-0.953535,0.946532,1.265623,0.679602,-0.800586,0.443622,-0.393545,...,0.984387,0.138978,0.457529,0.083267,0.350781,1.165338,0.589153,-1.677578,1.209412,-1.430639


In [101]:
train.to_csv('train.csv')
test.to_csv('test.csv')